In [1]:
import pandas as pd
from sklearn import linear_model

## 讀取資料

In [2]:
train_data = pd.read_csv("kc_house_train_data.csv")
test_data = pd.read_csv("kc_house_test_data.csv")

## 抓取特徵

In [3]:
train_features = train_data[['sqft_living',  'bedrooms', 'bathrooms']]
train_results = train_data[["price"]]

## 開始訓練模型

In [4]:
model = linear_model.LinearRegression()
model.fit(X = train_features,y = train_results)
print ("model" , model.coef_)

model [[   315.40669062 -65081.88711588   6942.16598637]]


## 預測
有一點誤差是正常的

In [5]:
model.predict(train_data[['sqft_living',  'bedrooms', 'bathrooms']])[0] #should be 271789.505878

array([ 271789.26537997])

## 計算RSS

In [6]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)
    # Then compute the residuals/errors
    residuals = outcome - predictions
    # Then square and add them up
    RSS = residuals.apply(lambda x: x**2).sum()
    return(RSS)

## 看看RSS對不對
有一點誤差是正常的

In [7]:
rss_example_train = get_residual_sum_of_squares(model, test_data[['sqft_living',  'bedrooms', 'bathrooms']], test_data[['price']])
print (rss_example_train) # should be 2.7376153833e+14

price    2.737619e+14
dtype: float64


## 建立一些新的特徵

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.
You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [8]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [9]:
# bedrooms_squared
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

# bed_bath_rooms
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms']* test_data['bathrooms']

# log_sqft_living
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

# lat_plus_long
train_data['lat_plus_long'] = train_data['long'] + train_data['lat']
test_data['lat_plus_long'] = test_data['long']+ test_data['lat']

## 訓練三個模型
Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']

model_2_features = model_1_features + ['bed_bath_rooms']

model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']


In [10]:
# model 1
model1 = linear_model.LinearRegression()
model1.fit(X = train_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']],y = train_data[['price']])

# model 2
model2 = linear_model.LinearRegression()
model2.fit(X = train_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long','bed_bath_rooms']],y = train_data[['price']])

# model 3
model3 = linear_model.LinearRegression()
model3.fit(X = train_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long','bedrooms_squared', 'log_sqft_living', 'lat_plus_long']],y = train_data[['price']])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [11]:
# Examine/extract each model's coefficients:
print ("model1" , model1.coef_)
print ("model2" , model2.coef_)
print ("model3" , model3.coef_)

model1 [[  3.12258646e+02  -5.95865332e+04   1.57067421e+04   6.58619264e+05
   -3.09374351e+05]]
model2 [[  3.06610053e+02  -1.13446368e+05  -7.14613083e+04   6.54844630e+05
   -2.94298969e+05   2.55796520e+04]]
model3 [[  5.17895745e+02   3.47434918e+04   3.64681376e+04   5.34373572e+05
   -4.06758960e+05  -9.44734086e+03  -5.36751710e+05   1.27614612e+05]]


In [12]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
RSS1 = get_residual_sum_of_squares(model1, train_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']], train_data[['price']])
print ("RSS1",RSS1) 

RSS2 = get_residual_sum_of_squares(model2, train_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long','bed_bath_rooms']], train_data[['price']])
print ("RSS1",RSS2) 

RSS2 = get_residual_sum_of_squares(model3, train_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long','bedrooms_squared', 'log_sqft_living', 'lat_plus_long']], train_data[['price']])
print ("RSS1",RSS2) 

RSS1 price    9.678800e+14
dtype: float64
RSS1 price    9.584196e+14
dtype: float64
RSS1 price    9.039047e+14
dtype: float64


In [13]:
# Compute the RSS on TESTING data for each of the three models and record the values:
# Compute the RSS on TRAINING data for each of the three models and record the values:
test_RSS1 = get_residual_sum_of_squares(model1, test_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']], test_data[['price']])
print ("test_RSS1",test_RSS1) 

test_RSS2 = get_residual_sum_of_squares(model2, test_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long','bed_bath_rooms']], test_data[['price']])
print ("test_RSS1",test_RSS2) 

test_RSS2 = get_residual_sum_of_squares(model3, test_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long','bedrooms_squared', 'log_sqft_living', 'lat_plus_long']], test_data[['price']])
print ("test_RSS1",test_RSS2) 

test_RSS1 price    2.255005e+14
dtype: float64
test_RSS1 price    2.233775e+14
dtype: float64
test_RSS1 price    2.979355e+14
dtype: float64


## Quiz Question:
What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)

In [14]:
print (test_data['bedrooms_squared'].mean())
print (test_data['bed_bath_rooms'].mean())
print (test_data['log_sqft_living'].mean())
print (test_data['lat_plus_long'].mean())

12.4466777015843
7.5039016315913925
7.550274679645921
-74.65333355403185


6.Quiz Question: What is the sign (positive or negative) for the coefficient/weight for ‘bathrooms’ in Model 1?

7.Quiz Question: What is the sign (positive or negative) for the coefficient/weight for ‘bathrooms’ in Model 2?

8.Is the sign for the coefficient the same in both models? Think about why this might be the case.

9.Now using your three estimated models compute the RSS (Residual Sum of Squares) on the Training data.

10.Quiz Question: Which model (1, 2 or 3) had the lowest RSS on TRAINING data?

11.Now using your three estimated models compute the RSS on the Testing data

12.Quiz Question: Which model (1, 2, or 3) had the lowest RSS on TESTING data?

13.Did you get the same answer for 9 and 11? Think about why this might be the case.